## Project Name: Ethnicity
Developed By: Nishant Sharma
### Project Summary:

### Objective:
Create a model to predict the nationality of a person from an uploaded image, along with predicting their emotion.

### Specifications:

* Indian Nationality: Predict age, dress color, and emotion.
* United States Nationality: Predict age and emotion.
* African Nationality: Predict emotion and dress color.
* Other Nationalities: Predict nationality and emotion.

### Constraints:
The model should reject ages below 10 and above 60, working only for ages in between.

This project aims to develop a comprehensive model for nationality and emotion prediction based on uploaded images, with specific features predicted based on the nationality identified.








#### Code Block 1: Load the Data
This block of code will help in loading the images and their respective labels from the dataset directory

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence

# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Define the ImageDataGenerator
class ImageDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE):
        valid_indices = [i for i, label in enumerate(labels) if -1 not in label]
        self.image_paths = [image_paths[i] for i in valid_indices]
        self.labels = [labels[i] for i in valid_indices]
        self.batch_size = batch_size
        self.image_size = image_size
        self.indices = np.arange(len(self.image_paths))

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_image_paths = [self.image_paths[i] for i in batch_indices]
        batch_labels = [self.labels[i] for i in batch_indices]
        images = []
        batch_labels_dicts = {'nationality': [], 'age': [], 'emotion': [], 'dress_color': []}
        for idx, img_path in enumerate(batch_image_paths):
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, self.image_size)
                img = img / 255.0
                images.append(img)
                label = batch_labels[idx]
                batch_labels_dicts['nationality'].append(label[0])
                batch_labels_dicts['age'].append(label[1])
                batch_labels_dicts['emotion'].append(label[2])
                batch_labels_dicts['dress_color'].append(label[3])
        return np.array(images), {key: np.array(value) for key, value in batch_labels_dicts.items()}

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

# Function to load images from directory with subfolders representing categories
def load_images_from_directory_with_subfolders(directory, label_template):
    image_paths = []
    labels = []
    for subfolder in os.listdir(directory):
        subfolder_path = os.path.join(directory, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            if os.path.isfile(img_path):
                image_paths.append(img_path)
                labels.append(label_template[int(subfolder)])  # Convert subfolder name to int and use as index for label
    return image_paths, labels

# Function to load images from directory without specific subfolder logic
def load_images_simple(directory, label):
    image_paths = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            image_paths.append(img_path)
            labels.append(label)
    return image_paths, labels

# Load and combine datasets
base_path = r"C:\Users\nisha\OneDrive\Documents\Ethnicity Dataset"
all_image_paths = []
all_labels = []

# AffectNet
affectnet_path = os.path.join(base_path, "AffectNet")
for folder in ["train", "val", "test"]:
    folder_path = os.path.join(affectnet_path, folder)
    paths, labels = load_images_from_directory_with_subfolders(folder_path, [[i, -1, -1, -1] for i in range(8)])
    all_image_paths.extend(paths)
    all_labels.extend(labels)

# Audience
audience_path = os.path.join(base_path, "Audience")
paths, labels = load_images_simple(audience_path, [0, -1, -1, -1])
all_image_paths.extend(paths)
all_labels.extend(labels)

# DeepFashion
deepfashion_path = os.path.join(base_path, "DeepFashion")
for category in ["Men", "Women"]:
    category_path = os.path.join(deepfashion_path, category)
    for item in os.listdir(category_path):
        item_path = os.path.join(category_path, item)
        paths, labels = load_images_simple(item_path, [1, -1, -1, -1])
        all_image_paths.extend(paths)
        all_labels.extend(labels)

# IMFD
imfd_path = os.path.join(base_path, "IMFD")
for folder in ["Test", "Train"]:
    folder_path = os.path.join(imfd_path, folder)
    paths, labels = load_images_simple(folder_path, [2, -1, -1, -1])
    all_image_paths.extend(paths)
    all_labels.extend(labels)

# RAF-DB
raf_db_path = os.path.join(base_path, "RAF-DB")
for folder in ["test", "train"]:
    folder_path = os.path.join(raf_db_path, folder)
    paths, labels = load_images_from_directory_with_subfolders(folder_path, [[3, -1, -1, -1] for i in range(8)])
    all_image_paths.extend(paths)
    all_labels.extend(labels)

# UTKFace
utkface_path = os.path.join(base_path, "UTKFace")
paths, labels = load_images_simple(utkface_path, [0, 25, 1, 2])
all_image_paths.extend(paths)
all_labels.extend(labels)

# Create data generators
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(all_image_paths, all_labels, test_size=0.2, random_state=42)
train_generator = ImageDataGenerator(train_image_paths, train_labels)
val_generator = ImageDataGenerator(val_image_paths, val_labels)


#### Code Block 2: Model Development

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model():
    input_layer = Input(shape=(224, 224, 3))

    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)

    # Fully connected layers
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output layers
    nationality_output = Dense(4, activation='softmax', name='nationality')(x)  # Assuming 4 nationalities
    age_output = Dense(1, activation='linear', name='age')(x)  # Regression for age
    emotion_output = Dense(4, activation='softmax', name='emotion')(x)  # Assuming 4 emotions
    dress_color_output = Dense(4, activation='softmax', name='dress_color')(x)  # Assuming 4 dress colors

    model = Model(inputs=input_layer, outputs=[nationality_output, age_output, emotion_output, dress_color_output])

    model.compile(optimizer='adam', 
                  loss={
                      'nationality': 'sparse_categorical_crossentropy',
                      'age': 'mse',
                      'emotion': 'sparse_categorical_crossentropy',
                      'dress_color': 'sparse_categorical_crossentropy'
                  },
                  metrics={
                      'nationality': 'accuracy', 
                      'age': 'mae', 
                      'emotion': 'accuracy', 
                      'dress_color': 'accuracy'
                  })

    model.summary()
    return model

model = create_model()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 222, 222,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 111, 111,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 54, 54,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 52, 52,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 26, 26,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 86528)     │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │ 44,302,848 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ nationality (Dense) │ (None, 4)         │      2,052 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ age (Dense)         │ (None, 1)         │        513 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emotion (Dense)     │ (None, 4)         │      2,052 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dress_color (Dense) │ (None, 4)         │      2,052 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 44,402,765 (169.38 MB)

 Trainable params: 44,402,765 (169.38 MB)

 Non-trainable params: 0 (0.00 B)

#### Code Block 3: Training the Model

In [3]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)


Epoch 1/20


c:\Users\nisha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


596/596 ━━━━━━━━━━━━━━━━━━━━ 250s 415ms/step - age_mae: 2.8713 - dress_color_accuracy: 0.9643 - emotion_accuracy: 0.9844 - loss: 21.8489 - nationality_accuracy: 0.9730 - val_age_mae: 0.1827 - val_dress_color_accuracy: 1.0000 - val_emotion_accuracy: 1.0000 - val_loss: 0.0565 - val_nationality_accuracy: 1.0000
Epoch 2/20
596/596 ━━━━━━━━━━━━━━━━━━━━ 243s 408ms/step - age_mae: 1.5623 - dress_color_accuracy: 1.0000 - emotion_accuracy: 1.0000 - loss: 3.7899 - nationality_accuracy: 1.0000 - val_age_mae: 0.4536 - val_dress_color_accuracy: 1.0000 - val_emotion_accuracy: 1.0000 - val_loss: 0.2082 - val_nationality_accuracy: 1.0000
Epoch 3/20
596/596 ━━━━━━━━━━━━━━━━━━━━ 252s 422ms/step - age_mae: 1.5135 - dress_color_accuracy: 1.0000 - emotion_accuracy: 1.0000 - loss: 3.6498 - nationality_accuracy: 1.0000 - val_age_mae: 0.1381 - val_dress_color_accuracy: 1.0000 - val_emotion_accuracy: 1.0000 - val_loss: 0.0202 - val_nationality_accuracy: 1.0000
Epoch 4/20
596/596 ━━━━━━━━━━━━━━━━━━━━ 254s 425ms

#### Code Block 4: Evaluation and Loading Up the Model

In [4]:
def evaluate_model(model, test_generator):
    results = model.evaluate(test_generator, verbose=1)
    print("Test Loss, Test Accuracy:", results)


In [6]:
# Save the model in the newer .keras format
model.save('path_to_my_model.keras')

# Load the model
model = tf.keras.models.load_model('path_to_my_model.keras')

from tensorflow.keras.metrics import MeanSquaredError

model = tf.keras.models.load_model(
    'path_to_my_model.keras',
    custom_objects={'mse': MeanSquaredError}
)


c:\Users\nisha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 18 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#### Code Block 5: Building a GUI for Emotion Prediction

In [11]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('path_to_my_model.keras')

def load_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Load and display the image
        img = Image.open(file_path)
        img.thumbnail((224, 224))  # Resize the image to fit in the display area
        img_display = ImageTk.PhotoImage(img)
        panel.configure(image=img_display)
        panel.image = img_display

        # Prepare the image for prediction
        img = img.resize((224, 224))
        img_array = np.array(img)
        img_array = img_array / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Predict using the model
        predictions = model.predict(img_array)
        results_text = f"Nationality: {np.argmax(predictions[0])}\n" \
                       f"Age: {predictions[1][0][0]:.2f}\n" \
                       f"Emotion: {np.argmax(predictions[2])}\n" \
                       f"Dress Color: {np.argmax(predictions[3])}"
        results_label.config(text=results_text)

# Set up the main window
root = tk.Tk()
root.title("Image Prediction")

# Create a panel to display the image
panel = tk.Label(root)
panel.pack()

# Create a button to load the image
load_button = tk.Button(root, text="Load Image", command=load_image)
load_button.pack()

# Create a label to display the predictions
results_label = tk.Label(root, text="", pady=20)
results_label.pack()

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
